In [ ]:
#!pip install imageio.v3
#!pip install mplsoccer

# Import statements
import os
import numpy as np
import pandas as pd
from matplotlib import animation, patches, pyplot as plt
from mplsoccer import Pitch
import imageio.v3 as imageio

In [ ]:
# Params
GAME_DURATION_SECONDS = 180
START_TIME = 0
FRAME_INCREMENT = 5
PITCH_WIDTH = 68
PITCH_LENGTH = 105
HALF_PITCH = True

In [ ]:
# Function to load data
def load_data(link):
    df = pd.read_csv(link, skiprows=2)
    df.sort_values('Time [s]', inplace=True)
    return df

# Load data
link_away = ('https://raw.githubusercontent.com/metrica-sports/sample-data/master/'
             'data/Sample_Game_1/Sample_Game_1_RawTrackingData_Away_Team.csv')
link_home = ('https://raw.githubusercontent.com/metrica-sports/sample-data/master/'
             'data/Sample_Game_1/Sample_Game_1_RawTrackingData_Home_Team.csv')
df_away = load_data(link_away)
df_home = load_data(link_home)

#display(df_away)


# Reset column names
def set_col_names(df):
    """Renames the columns to have x and y suffixes."""
    cols = list(np.repeat(df.columns[3::2], 2))
    cols = [col+'_x' if i % 2 == 0 else col+'_y' for i, col in enumerate(cols)]
    cols = np.concatenate([df.columns[:3], cols])
    df.columns = cols

set_col_names(df_away)
set_col_names(df_home)

# Subset data
end_time = START_TIME + GAME_DURATION_SECONDS
df_away = df_away[(df_away['Time [s]'] >= START_TIME) & (df_away['Time [s]'] < end_time)].copy()
df_home = df_home[(df_home['Time [s]'] >= START_TIME) & (df_home['Time [s]'] < end_time)].copy()

# Split off ball data
df_ball = df_away[['Period', 'Frame', 'Time [s]', 'Ball_x', 'Ball_y']].copy()
df_home.drop(['Ball_x', 'Ball_y'], axis=1, inplace=True)
df_away.drop(['Ball_x', 'Ball_y'], axis=1, inplace=True)
df_ball.rename({'Ball_x': 'x', 'Ball_y': 'y'}, axis=1, inplace=True)

# Convert to long form
def to_long_form(df):
    """Pivots a dataframe from wide-form to long form."""
    df = pd.melt(df, id_vars=df.columns[:3], value_vars=df.columns[3:], var_name='player')
    df.loc[df.player.str.contains('_x'), 'coordinate'] = 'x'
    df.loc[df.player.str.contains('_y'), 'coordinate'] = 'y'
    df = df.dropna(axis=0, how='any')
    df['player'] = df.player.str[6:-2]
    df = (df.set_index(['Period', 'Frame', 'Time [s]', 'player', 'coordinate'])['value']
          .unstack()
          .reset_index()
          .rename_axis(None, axis=1))
    return df

df_away = to_long_form(df_away)
df_home = to_long_form(df_home)

# Drop off unnecessary columns
df_home.drop(['Period', 'Time [s]', 'player'], axis=1, inplace=True)
df_away.drop(['Period', 'Time [s]', 'player'], axis=1, inplace=True)
df_ball.drop(['Period', 'Time [s]'], axis=1, inplace=True)

#display(df_away)
#display(df_ball)

# Filter out duplicate rows and decide which ones to plot with FRAME_INCREMENT
frames = list(set(df_away.Frame))
frames = frames[::FRAME_INCREMENT]

filenames = []


In [ ]:
def invert_coordinates(away_coordinates, home_coordinates, ball_coordinates):
    away_coordinates['x'] = 1 - away_coordinates['x']
    home_coordinates['x'] = 1 - home_coordinates['x']
    ball_coordinates['x'] = 1 - ball_coordinates['x']

    return away_coordinates, home_coordinates, ball_coordinates

def segment_pitch(away_coordinates, home_coordinates, ball_coordinates):
    away_team_in_opposite_side_count = (away_coordinates['x'] < 0.5).sum()
    home_team_in_opposite_side_count = (home_coordinates['x'] > 0.5).sum()

    if away_team_in_opposite_side_count > home_team_in_opposite_side_count:
        print("Away team opposite side count:", away_team_in_opposite_side_count,\
              "; Home team opposite side count:", home_team_in_opposite_side_count)
        return invert_coordinates(away_coordinates, home_coordinates, ball_coordinates)

    return away_coordinates, home_coordinates, ball_coordinates

In [ ]:
p = Pitch(pitch_type='metricasports', goal_type='line', half=HALF_PITCH,
          pitch_width=PITCH_WIDTH, pitch_length=PITCH_LENGTH,
          pad_left=None, pad_right=None, pad_bottom=None, pad_top=None,
          line_color=None, goal_alpha=0, line_alpha=0)


for frame in frames:
    fig, ax = p.draw()

    away_coordinates = df_away[df_away.Frame == frame]
    home_coordinates = df_home[df_home.Frame == frame]
    ball_coordinates = df_ball[df_ball.Frame == frame]

    away_coordinates, home_coordinates, ball_coordinates = \
        segment_pitch(away_coordinates, home_coordinates, ball_coordinates)

    player_coordinates = pd.concat([away_coordinates, home_coordinates])

    # Plot Voronoi
    team1, team2 = p.voronoi(player_coordinates.x, player_coordinates.y,
                              [0]*11 + [1]*11)

    t1 = p.polygon(team1, ax=ax, fc='orange', ec='black', lw=0, alpha=1)
    t2 = p.polygon(team2, ax=ax, fc='dodgerblue', ec='black', lw=0, alpha=1)

    # Plot players
    #sc1 = p.scatter(away_coordinates.x, away_coordinates.y, c='dodgerblue', s=80, ec='k', ax=ax)
    #sc2 = p.scatter(home_coordinates.x, home_coordinates.y, c='orange', s=80, ec='k', ax=ax)

    # Plot ball
    #sc3 = p.scatter(ball_coordinates.x, ball_coordinates.y, c='white', s=30, ec='k', ax=ax)

    # Create file name and append it to a list
    filename = f'/content/drive/MyDrive/Master 1/Projet TER/Images/Metrica Sports/Full Pitch/voronoi_{frame}.png'
    filenames.append(filename)

    # Save frame
    #plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    #plt.close()




In [ ]:
# Remove image files on drive
for filename in set(filenames):
    os.remove(filename)

In [ ]:
def generate_gif(filenames):
    # Generates a gif from the pre-generated temp images.
    # Build gif
    gif_name = f'Voronoi_{START_TIME}_{GAME_DURATION_SECONDS}_{FRAME_INCREMENT}.gif'
    images = []
    for filename in filenames:
       images.append(imageio.imread(filename))
    imageio.imwrite(gif_name, images, loop=0)

    # Remove temp image files
    for filename in set(filenames):
        os.remove(filename)


#generate_gif(filenames)

In [ ]:
# TESTS

print(away_coordinates)
away_coordinates.x = away_coordinates.x.map(lambda x: 1-x)
print(away_coordinates)
print(away_coordinates.x)
print(away_coordinates['x'])


       Frame        x        y
16555   1506  0.03437  0.28076
16556   1506  0.04796  0.38787
16557   1506  0.06597  0.47996
16558   1506  0.07699  0.33265
16559   1506  0.11313  0.41659
16560   1506  0.09116  0.49120
16561   1506  0.22736  0.54963
16562   1506  0.18101  0.58370
16563   1506  0.35980  0.55805
16564   1506  0.37221  0.43667
16565   1506  0.01806  0.44883
       Frame        x        y
16555   1506  0.96563  0.28076
16556   1506  0.95204  0.38787
16557   1506  0.93403  0.47996
16558   1506  0.92301  0.33265
16559   1506  0.88687  0.41659
16560   1506  0.90884  0.49120
16561   1506  0.77264  0.54963
16562   1506  0.81899  0.58370
16563   1506  0.64020  0.55805
16564   1506  0.62779  0.43667
16565   1506  0.98194  0.44883
16555    0.96563
16556    0.95204
16557    0.93403
16558    0.92301
16559    0.88687
16560    0.90884
16561    0.77264
16562    0.81899
16563    0.64020
16564    0.62779
16565    0.98194
Name: x, dtype: float64
16555    0.96563
16556    0.95204
16557    0.

<ipython-input-64-7d38383170bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_coordinates.x = away_coordinates.x.map(lambda x: 1-x)
